In [7]:
import os
from glob import glob
from tqdm import tqdm
import pandas as pd
from PIL import Image
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from efficientnet_pytorch import EfficientNet

In [14]:
# transform configuration
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.2, 0.2, 0.2])
])

In [ ]:
optim.lr_scheduler.CosineAnnealingLR()

In [50]:
model = VanillaEfficientNet(18)

Loaded pretrained weights for efficientnet-b3


In [51]:
temp = torch.rand(4,3,224,224)
temp.size()

torch.Size([4, 3, 224, 224])

In [52]:
model(temp).size()

torch.Size([4, 18])

In [44]:
m = nn.BatchNorm1d(100)
m = nn.BatchNorm1d(100, affine=False)
input = torch.randn(2, 100)
output = m(input)

In [51]:
# simple pretrained model
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(512, 18)

# freeze
for name, param in model.named_parameters():
    if name not in ['fc.weight', 'fc.bias']:
        param.requires_grad = False # freeze

In [53]:
# train configuration
EPOCHS = 3
BATCH_SIZE = 64
LR = 0.0005

In [54]:
# compile
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

In [59]:
# train phase
model.cuda()
model.train()

for epoch in range(EPOCHS):
    for imgs, labels in tqdm(loader, desc='Train'):
        imgs = imgs.cuda()
        labels = labels.cuda()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Train: 100%|██████████| 885/885 [01:43<00:00,  8.54it/s]


In [75]:
# validation phase
model.eval()

total_corrects = 0
total_samples = 0
with torch.no_grad():
    for imgs, labels in tqdm(validloader, desc='Validation'):
        imgs = imgs.cuda()
        
        outputs = model(imgs)
        _, pred = torch.max(outputs, 1)
        pred = pred.cpu().numpy()
        labels = labels.cpu().numpy()
        
        total_corrects += (labels == pred).sum()
        total_samples += len(pred)

Validation: 100%|██████████| 178/178 [00:21<00:00,  8.38it/s]


In [76]:
print(f'Accuracy: {total_corrects/total_samples:.4f}')

Accuracy: 0.7873


In [103]:
class EvalDataset(Dataset):
    def __init__(self, transform=None):
        img_id_list = pd.read_csv('../input/data/eval/info.csv')['ImageID'] # to keep id order corresponding to that of submission
        self.img_paths = list(map(lambda x: os.path.join('../input/data/eval/images', x), img_id_list))
        self.transform = transform

    def __getitem__(self, index):
        name = os.path.basename(self.img_paths[index]) # ImageID
        image = Image.open(self.img_paths[index]) # image array
        if self.transform:
            image = self.transform(image) # transform(optional)
        return name, image

    def __len__(self):
        return len(self.img_paths)

In [113]:
# inference phase

evalset = EvalDataset(transform)
evalloader = DataLoader(evalset, batch_size=64, shuffle=False, drop_last=False)

model.eval()

name_list = []
pred_list = []

with torch.no_grad():
    for names, imgs in tqdm(evalloader):
        imgs = imgs.cuda()
        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)
        
        pred_list.extend(preds.cpu().numpy().tolist())
        name_list.extend(names)

100%|██████████| 197/197 [01:26<00:00,  2.27it/s]


In [122]:
# check inference size
if len(name_list) == len(pred_list):
    print('Correct Inference Size:', len(name_list), len(pred_list))

Correct Inference Size: 12600 12600


In [125]:
# check inference order
submission_template = pd.read_csv('../input/data/eval/submission.csv')
id_order = submission_template['ImageID'].tolist()

allright = True
for i, j in zip(id_order, name_list):
    if i != j:
        allright = False
        print('Incorrect Order', i, j)

if allright:
    print('Correct ImageID Order')

Correct ImageID Order


In [117]:
# compose submission
submission = pd.DataFrame(dict(ImageID=name_list, ans=pred_list))
submission.to_csv('submission_please.csv', index=False)